In [1]:
import pyspark 
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf, SparkContext
import os
import re
import numpy as np
import time
from functools import reduce
import pandas as pd

Location_master = 'spark://linux:7077'
Location_master = 'local'
try:
    sparkSession.stop()
except:
    pass
sparkSession = SparkSession \
    .builder \
    .master(Location_master)\
    .appName("Process 2020") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "4g") \
    .config('spark.cores.max','4')\
    .getOrCreate()

In [2]:
data_dir = '../../../Data_process/'
data = sparkSession.read.csv(data_dir + 'Diemthi2021/*/', header=True)
data_bosung = sparkSession.read.csv(data_dir + 'CrawlBosung/missing.csv', header=True)

data.show(5)
data_bosung.show(5)

data.printSchema()
data_bosung.printSchema()

data_all = reduce(DataFrame.unionAll, [data, data_bosung])

+---+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
|_c0|     SBD|Cum_thi|Toan|Ngu_van|Ngoai_ngu|Vat_ly|Hoa_hoc|Sinh_hoc|KHTN|Lich_su|Dia_ly|GDCD|KHXH|
+---+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
|  0|02000096|     02| 7.8|   5.75|     null|  7.75|   6.75|     5.5|6.67|   null|  null|null|null|
|  1|02000095|     02|   8|   null|     null|  null|   null|    null|6.67|   null|  null|null|null|
|  2|02000094|     02|   8|   6.75|      8.8|   5.5|   6.25|    7.25|6.33|   null|  null|null|null|
|  3|02000092|     02| 8.8|   6.75|        9|  8.75|   7.75|    3.25|6.58|   null|  null|null|null|
|  4|02000091|     02|   8|   5.75|      9.2|  8.75|   8.25|     5.5| 7.5|   null|  null|null|null|
+---+--------+-------+----+-------+---------+------+-------+--------+----+-------+------+----+----+
only showing top 5 rows

+---+--------+-------+----+-------+---------+------+-------+--------+----+-

### Thay đổi kiểu dữ liệu các trường dữ liệu về đúng dạng của nó

In [3]:
import pyspark.sql.functions as F
import pyspark.sql.functions as f
from pyspark.sql.types import FloatType, StringType, IntegerType

In [4]:
data_all = data_all.drop('_c0')
data_all = data_all.withColumn('Toan', F.col('Toan').cast(FloatType()))
data_all = data_all.withColumn('Ngu_van', F.col('Ngu_van').cast(FloatType()))
data_all = data_all.withColumn('Ngoai_ngu', F.col('Ngoai_ngu').cast(FloatType()))
data_all = data_all.withColumn('Vat_ly', F.col('Vat_ly').cast(FloatType()))
data_all = data_all.withColumn('Hoa_hoc', F.col('Hoa_hoc').cast(FloatType()))
data_all = data_all.withColumn('Sinh_hoc', F.col('Sinh_hoc').cast(FloatType()))
data_all = data_all.withColumn('Lich_su', F.col('Lich_su').cast(FloatType()))
data_all = data_all.withColumn('Dia_ly', F.col('Dia_ly').cast(FloatType()))
data_all = data_all.withColumn('GDCD', F.col('GDCD').cast(FloatType()))

data_all.printSchema()

root
 |-- SBD: string (nullable = true)
 |-- Cum_thi: string (nullable = true)
 |-- Toan: float (nullable = true)
 |-- Ngu_van: float (nullable = true)
 |-- Ngoai_ngu: float (nullable = true)
 |-- Vat_ly: float (nullable = true)
 |-- Hoa_hoc: float (nullable = true)
 |-- Sinh_hoc: float (nullable = true)
 |-- KHTN: string (nullable = true)
 |-- Lich_su: float (nullable = true)
 |-- Dia_ly: float (nullable = true)
 |-- GDCD: float (nullable = true)
 |-- KHXH: string (nullable = true)



In [21]:
data_check = data_all.withColumn('crawl', f.substring(f.col('SBD'), 3, 4)).select('Cum_thi', 'crawl')
data_check = data_check.groupBy('Cum_thi', 'crawl').count().orderBy('Cum_thi', 'crawl').show()

+-------+-----+-----+
|Cum_thi|crawl|count|
+-------+-----+-----+
|     01| 0000|   97|
|     01| 0001|  100|
|     01| 0002|  100|
|     01| 0003|  100|
|     01| 0004|   98|
|     01| 0005|   98|
|     01| 0006|   99|
|     01| 0007|   99|
|     01| 0008|  100|
|     01| 0009|   99|
|     01| 0010|  100|
|     01| 0011|   99|
|     01| 0012|   98|
|     01| 0013|  100|
|     01| 0014|  100|
|     01| 0015|  100|
|     01| 0016|   98|
|     01| 0017|  100|
|     01| 0018|  100|
|     01| 0019|  100|
+-------+-----+-----+
only showing top 20 rows



In [15]:
data_check

,Cum_thi,crawl
0,01,020623
1,01,028260
2,01,029544
3,01,046741
4,01,057709
...,...,...
923135,44,012903
923136,51,000390
923137,54,011338
923138,56,005440


In [7]:
from pyspark.sql.window import Window

data_all = data_all.withColumn('row_num', f.row_number().over(
    Window.partitionBy('SBD').orderBy('SBD')
))

In [8]:
# if(data_bosung.count() == 0):
#     data_dup = data_all.filter('row_num = 2').withColumn('raw', f.substring(f.col('SBD'),0,5))
#     data_dup = data_dup.groupBy('raw').count().toPandas()
#     list_data_miss = []
#     list_dup = list(data_dup['raw'].values)
#     for dup in list_dup:
#         for i in range(0,10):
#             pattern = dup + str(i)
#             if(data_all.filter("SBD like '{0}%'".format(pattern)).count() == 0):
#                 list_data_miss.append(pattern)

#     list_data_miss = pd.DataFrame({'Miss': list_data_miss})
#     list_data_miss.to_csv("../../../Data_process/ListMiss/miss.csv")

In [9]:
print(data_all.count())
data_all = data_all.filter('row_num == 1')
data_all = data_all.drop("row_num")
data_all = data_all.drop_duplicates()
print(data_all.count())
data_all = data_all.orderBy("SBD")
data_all.repartition(1).write.format("com.databricks.spark.csv")\
        .option("header", "true").save("mydata.csv", mode='overwrite')

938487
923140


In [10]:
# sparkSession.stop()